# Naive Bayes

Here is a step by step explanation of the algorithm: https://youtu.be/O2L2Uv9pdDA

## Measuring accuracy

Just as for the Decision Tree algorithm, accuracy in this case is defined as the proportion of correctly classified instances.
For _training accuracy_ we can use the dedicated score function:

In [1]:
# Prepare the data as usual
import pandas as pd
features = ['study', 'free', 'money']
target = 'is_spam'
messages = pd.DataFrame(
  [(1, 0, 0, 0),
  (0, 0, 1, 0),
  (1, 0, 0, 0),
  (1, 1, 0, 0)] +
  [(0, 1, 0, 1)] * 4 +
  [(0, 1, 1, 1)] * 4,
columns=features+[target])
messages

# Fit a Naive Bayes classifier
from sklearn.naive_bayes import BernoulliNB
X = messages[features]
y = messages[target]
cl = BernoulliNB().fit(X, y)

# Measure accuracy on the training set
cl.score(X, y)

0.9166666666666666

Again, this is the same as predicting the training set and then counting how many
predictions actually match the labels:

In [2]:
y_hat = cl.predict(X)
(y_hat == y).mean()

0.9166666666666666

Similarly, _cross-validation leave one out_ (_CVLOO_) works in the same way as for Decision Trees:

In [3]:
from sklearn.model_selection import LeaveOneOut, cross_val_score
from statistics import mean
loo = LeaveOneOut()
scores = cross_val_score(cl, X, y, cv=loo)
print("CVLOO scores:", scores)
print("Mean CVLOO score: ", mean(scores))

CVLOO scores: [1. 0. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1.]
Mean CVLOO score:  0.8333333333333334


## Correlated features

The naive assumption in the naive Bayes algorithm is that the features are independent. We can see what happens when this assumption is not true by adding a new feature to our dataset that is highly correlated with an existing one.

For example, if we also consider the word ”school”, which appears roughly in the same message as the word ”study” and we want to predict "study money" we end up with the following dataset:

In [4]:
new_messages = pd.DataFrame(
  [(1, 0, 1)],
columns = features)

new_messages_dep = new_messages.copy()
new_messages_dep['school'] = new_messages['study']
X_dep = X.copy()
X_dep['school'] = X['study']
X_dep

,study,free,money,school
0,1,0,0,1
1,0,0,1,0
2,1,0,0,1
3,1,1,0,1
4,0,1,0,0
5,0,1,0,0
6,0,1,0,0
7,0,1,0,0
8,0,1,1,0
9,0,1,1,0


Now let's compare the prediction when "school" is used and when "school" is not used as a training word. When the dependent variable is present:

In [5]:
# Correlated variable is present
cl = BernoulliNB().fit(X_dep, y)
cl.predict_proba(new_messages_dep)

array([[0.98997649, 0.01002351]])

And when it is not present:

In [6]:
# Correlated variable not present
cl = BernoulliNB().fit(X, y)
cl.predict_proba(new_messages)

array([[0.93676815, 0.06323185]])

The probability of being spam decreased to 0.01 from 0.063 since the presence of the word ”school” also contributes toward this new probability. This can lead to incorrect predictions since ”school” brings no new information as long as ”study” is already considered, so should not have changed the prediction.

We can compare this with the behaviour of ID3. When the dependent variable is present:

In [7]:
# Correlated variable is present
from sklearn import tree
dt = tree.DecisionTreeClassifier(criterion='entropy').fit(X_dep,y)
dt.predict_proba(new_messages_dep)

array([[1., 0.]])

And when it is not present:

In [8]:
# Correlated variable not present
dt = tree.DecisionTreeClassifier(criterion='entropy').fit(X,y)
dt.predict_proba(new_messages)

array([[1., 0.]])

As expected, ID3 is not influenced by the cloned variable.

## The Laplace estimator

Apart from the naivety assumption, there is another phenomenon affecting the predictions of Naive Bayes, when rare words are encountered. For instance, we saw that the message "study money" is predicted as non-spam with 93% probability, which seems reasonable. But what if we need to predict the message "study money university", and the word "university" has never been seen in a spam message before, only in regular messages?

If we look at the naive Bayes formula $ v_{MAP} = \mbox{argmax}_{v_j \in V} \prod_i P(a_i|v_j)P(v_j)$, we can see that all the terms are multiplied, while the term $P(\mbox{university}|\mbox{spam})$ is equal to 0, which makes the entire probability of "spam" become exactly 0.

This is unrealistic since there are many rare or misspelled words which can turn a prediction to 0 instantly, completely cancelling the effect of all the other words.

The Python implementation however, doesn't seem to give a 0% chance of spam:

In [9]:
# Prepare the data as usual
import pandas as pd
features = ['study', 'free', 'money', 'university']
target = 'is_spam'
messages = pd.DataFrame(
  [(1, 0, 0, 0, 0),
  (0, 0, 1, 1, 0),
  (1, 0, 0, 0, 0),
  (1, 1, 0, 0, 0)] +
  [(0, 1, 0, 0, 1)] * 4 +
  [(0, 1, 1, 0, 1)] * 4,
columns=features+[target])
messages

# Fit a Naive Bayes classifier
from sklearn.naive_bayes import BernoulliNB
X = messages[features]
y = messages[target]
cl = BernoulliNB().fit(X, y)

# Predict the message "study money university"
new_messages = pd.DataFrame(
  [(1, 0, 1, 1)],
columns = features)
cl.predict_proba(new_messages)

array([[0.98015192, 0.01984808]])

We can see the prediction for "study money university" is about 1.98%, even if "university" never shows up in a spam message. This is because `BernoulliNB()` uses a technique called _additive smoothing_ or _Laplace smoothing_, controlled through the `alpha` parameter. It essentially adds a constant value (by default 1) to each variable count, as if each variable was seen once for every value of the target. This makes sure that no probability becomes exactly 0. The Python implementation does not allow us to completely disable the estimator, but we can see what happens if we set it to something very close to 0:

In [10]:
cl = BernoulliNB(alpha=1e-10).fit(X, y)
cl.predict_proba(new_messages)

array([[1.00000000e+00, 5.55545733e-32]])

By disabling the Laplace smoothing, the predictions are now 100% and 0% respectively.

## Joint Bayes

The independence assumption of naive Bayes could theoretically be dropped, creating an algorithm usually named _Joint Bayes_, which is simply looking for the value 

$$ v_{MAP} = \mbox{argmax}_{v_j \in V} P(a_1,a_2,...,a_n|v_j)P(v_j) $$

Due to the exponential size of the model, this algorithm is not very practical, and consequently not available in the `scikit` library.

Since the attributes are no longer considered independent, the conditional probability $P(a_1,a_2,...,a_n|v_j)$ can no longer be transformed to $\prod_i P(a_i|v_j)P(v_j)$. If applied to a text dataset for instance, $n$ would be equal the number of words in a language and the model would have to learn all possible combinations of words in spam and non-spam, so $2 \times 2^n = 2^{n+1}$. Since English has about $n=170,000$ words in current use, we can see how this approach quickly becomes unfeasable.